# DSC148 Kaggle Data Challenge
Name: Yishan Cai

## Set up

In [1]:
import pandas as pd
import numpy as np

from math import isnan
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df = pd.read_csv('train.csv')
df.head(3)

/tmp/ipykernel_497/2051523488.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv')


,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,22267382,Modern and Cozy Large Studio in Brooklyn,Modern large studio with new amenities and app...,Our place is a little quiet sanctuary in the h...,Modern large studio with new amenities and app...,none,"BAM, Barclays, Brooklyn City Point, Fort Green...",NaN,"Subway: 2,3,4,5,A,C,B,Q,G",Washer/Dryer Dishwasher Internet Gym Roof Top ...,...,10.0,10.0,10.0,t,f,flexible,f,f,1,0.59
1,2473861,Royal Harlem TRIPLEX Home 5 Beds,Harlem is back and so gorgeous! Visit and expl...,Harlem is back and so gorgeous! Visit and expl...,Harlem is back and so gorgeous! Visit and expl...,none,HARLEM is a piece of real NY history overflowi...,HARLEM RESTAURANTS Red Rooster Harlem -- excel...,PUBLIC TRANSPORTATION: Conveniently near all p...,The WHOLE ENTIRE HOUSE,...,9.0,9.0,9.0,t,f,moderate,f,f,3,2.47
2,25079703,Sunny East Village Studio,"Clean, hip and well designed sun drenched East...",This is a rare East Village studio with it's h...,"Clean, hip and well designed sun drenched East...",none,East Village is one of the last remaining neig...,NaN,NaN,You'll have access to the entire space - it's ...,...,10.0,10.0,10.0,f,f,moderate,f,f,1,0.89


In [3]:
# number of observations
df.shape[0]

33538

In [4]:
df_pp = df.copy()

## one hot amenities & verifications

In [5]:
df_pp['host_since_year'] = pd.to_datetime(df_pp['host_since']).dt.year
df_pp['host_since_month'] = pd.to_datetime(df_pp['host_since']).dt.month
df_pp['host_since_day'] = pd.to_datetime(df_pp['host_since']).dt.day

In [6]:
df_pp['first_re_year'] = pd.to_datetime(df_pp['first_review']).dt.year
df_pp['first_re_month'] = pd.to_datetime(df_pp['first_review']).dt.month
df_pp['first_re_day'] = pd.to_datetime(df_pp['first_review']).dt.day
df_pp['last_re_year'] = pd.to_datetime(df_pp['last_review']).dt.year
df_pp['last_re_month'] = pd.to_datetime(df_pp['last_review']).dt.month
df_pp['last_re_day'] = pd.to_datetime(df_pp['last_review']).dt.day

In [7]:
all_amenities = set()
for amenities_str in df_pp['amenities']:
    amenities_list = amenities_str.strip('{}').split(',')
    for amenity in amenities_list:
        clean = amenity.strip().strip('"')
        all_amenities.add(clean)

all_amenities.discard('')

In [8]:
new_columns_data = {}
for idx, amenity in enumerate(all_amenities, start=1):
    new_columns_data[f'amenity_{idx}'] = df_pp['amenities'].apply(lambda x: 1 if amenity in x else 0)

df_pp = pd.concat([df_pp, pd.DataFrame(new_columns_data)], axis=1)

In [9]:
all_verifications = set()
for verifications_str in df_pp['host_verifications']:
    if isinstance(verifications_str, str):
        verifications_list = verifications_str.strip('[]').split(',')
        for verification in verifications_list:
            clean = verification.strip().strip("'")
            all_verifications.add(clean)

all_verifications.discard('')

veri_columns_data = {}
for idx, veri in enumerate(all_verifications, start=1):
    veri_columns_data[f'veri_{idx}'] = df_pp['host_verifications'].apply(lambda x: 1 if (isinstance(x, str) and veri in x) 
                                                                         else 0 if pd.isnull(x) else 0)

df_pp = pd.concat([df_pp, pd.DataFrame(veri_columns_data)], axis=1)

## Pre-defined functions

In [10]:
def to_float(s):
    try:
        ret = float(s)
    except:
        ret = -1
    if isnan(ret):
        ret = -1
    return ret

def to_int(s):
    try:
        ret = int(s)
    except:
        ret = -1
    return ret

def to_len(s):
    try:
        ret = len(s)
    except:
        ret = 0
    if isnan(ret):
        ret = 0
    return ret

def to_label(s):
    if s is None:
        ret = 0
    else:
        ret = 1
    return ret

def to_num(value):
    if pd.isna(value):
        return -1
    else:
        return int(value[:-1])

def to_tf_label(s):
    if s is None:
        ret = -1
    elif s == 't':
        ret = 1
    else: 
        ret = 0
    return ret

In [11]:
ng_cate = [x for x in list(set(df['neighbourhood_group_cleansed'])) if type(x) is str]
neighbor_cate = [x for x in list(set(df['neighbourhood_cleansed'])) if type(x) is str]
room_cate = [x for x in list(set(df['room_type'])) if type(x) is str]
cancel_cate = [x for x in list(set(df['cancellation_policy'])) if type(x) is str]
property_cate = [x for x in list(set(df['property_type'])) if type(x) is str]
bed_cate = [x for x in list(set(df['bed_type'])) if type(x) is str]

In [12]:
def create_feature(row):
    
    len_name = to_len(row.name)
    len_summary = to_len(row.summary)
    len_space = to_len(row.space)
    len_description = to_len(row.description)
    len_notes = to_len(row.notes)
    len_nearby = to_len(row.neighborhood_overview)
    len_transit = to_len(row.transit)
    
    access = to_label(row.access)
    interact = to_label(row.interaction)
    rule = to_label(row.house_rules)
    len_hname = to_len(row.host_name)
    len_habout = to_len(row.host_about)
    host_response = to_num(row.host_response_rate)
    host_loco = to_label(row.host_location)
    superhost = to_tf_label(row.host_is_superhost)
    host_pic = to_tf_label(row.host_has_profile_pic)
    host_veri = to_tf_label(row.host_identity_verified)
    n_list = to_int(row.host_listings_count)
    n_clist = to_int(row.calculated_host_listings_count)
    zipcode = to_int(row.zipcode)
    host_join_year = to_int(row.host_since_year)
    host_join_month = to_int(row.host_since_month)
    host_join_day= to_int(row.host_since_day)
    
    guest_num = to_int(row.guests_included)
    guest_capa = to_int(row.extra_people)
    n_bedroom = to_int(row.bedrooms)
    n_bed = to_int(row.beds)
    n_bathroom = to_int(row.bathrooms)
    n_acco = to_int(row.accommodates)
    
    n_min_night = to_int(row.minimum_nights)
    n_max_night = to_int(row.maximum_nights)
    n_review = to_int(row.number_of_reviews)
    
    re_rate = to_float(row.review_scores_rating)
    re_accu = to_float(row.review_scores_accuracy)
    re_clean = to_float(row.review_scores_cleanliness)
    re_checkin = to_float(row.review_scores_checkin)
    re_comm = to_float(row.review_scores_communication)
    re_loca = to_float(row.review_scores_location)
    re_val = to_float(row.review_scores_value)
    re_per_month = to_float(row.reviews_per_month)
    
    first_re_year = to_int(row.first_re_year)
    last_re_year = to_int(row.last_re_year)
    first_re_month = to_int(row.first_re_month)
    last_re_month = to_int(row.last_re_month)
    first_re_day = to_int(row.first_re_day)
    last_re_day = to_int(row.last_re_day)
    
    instant_book = to_tf_label(row.instant_bookable)
    busi_travel = to_tf_label(row.is_business_travel_ready)
    guest_pic = to_tf_label(row.require_guest_profile_picture)
    guest_veri = to_tf_label(row.require_guest_phone_verification)

    
    one_hot_ng = [int(row.neighbourhood_group_cleansed == category) for category in ng_cate]
    one_hot_neighbor = [int(row.neighbourhood_cleansed == category) for category in neighbor_cate]
    one_hot_room = [int(row.room_type == category) for category in room_cate]
    one_hot_bed = [int(row.bed_type == category) for category in bed_cate]
    one_hot_cancel = [int(row.cancellation_policy == category) for category in cancel_cate]
    one_hot_proper = [int(row.property_type == category) for category in property_cate]
    
    result = [len_name, len_summary, len_space, len_description, len_notes, len_nearby, len_transit, 
              access, interact, rule, len_hname, len_habout, host_response, host_loco, superhost, 
              host_pic, host_veri, n_list, n_clist, zipcode, host_join_year, host_join_month, host_join_day, 
              guest_num, guest_capa, n_bedroom, n_bed, n_bathroom, n_acco, n_review, n_min_night, n_max_night, 
              re_rate, re_accu, re_clean, re_checkin, re_comm, re_loca, re_val, re_per_month, 
              first_re_year, last_re_year, first_re_month, last_re_month, first_re_day, last_re_day,
              instant_book, busi_travel, guest_pic, guest_veri,
              row.amenity_1, row.amenity_2, row.amenity_3, row.amenity_4, row.amenity_8, row.amenity_12,
              row.amenity_13, row.amenity_14, row.amenity_17, row.amenity_18, 
              row.amenity_19, row.amenity_22, row.amenity_23, row.amenity_24, row.amenity_27, 
              row.amenity_32, row.amenity_33, row.amenity_34, row.amenity_35, row.amenity_36,
              row.amenity_37,row.amenity_39, row.amenity_40,row.amenity_42,
              row.amenity_45,row.amenity_46,row.amenity_47,row.amenity_48,
              row.amenity_49,row.amenity_50,row.amenity_52,row.amenity_54,
              row.amenity_58,row.amenity_59,row.amenity_60,
              row.amenity_62,row.amenity_65,row.amenity_66,
              row.amenity_67,row.amenity_68,row.amenity_70,row.amenity_71,row.amenity_72,
              row.amenity_73,row.amenity_74,row.amenity_76,row.amenity_77,row.amenity_78,
              row.amenity_80,row.amenity_82,row.amenity_84,
              row.amenity_85,row.amenity_86,row.amenity_87,row.amenity_89,row.amenity_90,
              row.amenity_91,row.amenity_93,row.amenity_94,row.amenity_95,
              row.amenity_97,row.amenity_98,row.amenity_100,row.amenity_101,row.amenity_102,
              row.amenity_104,row.amenity_106,row.amenity_107,
              row.amenity_109,row.amenity_112,row.amenity_114,
              row.amenity_115,row.amenity_116,row.amenity_117,row.amenity_118,row.amenity_119,row.amenity_120,
              row.amenity_122,row.amenity_123,row.amenity_124,
              row.amenity_5, row.amenity_6, row.amenity_7, row.amenity_9,row.amenity_10, row.amenity_11, 
              row.amenity_15, row.amenity_16, row.amenity_20, row.amenity_21, row.amenity_25, row.amenity_26, 
              row.amenity_28, row.amenity_29, row.amenity_30,row.amenity_31, row.amenity_38,row.amenity_41,
              row.amenity_43,row.amenity_44,row.amenity_51,row.amenity_53,row.amenity_55,row.amenity_56,
              row.amenity_57,row.amenity_61,row.amenity_63,row.amenity_64,row.amenity_69,row.amenity_75, 
              row.amenity_79,row.amenity_81,row.amenity_83, row.amenity_88, row.amenity_92,row.amenity_96,
              row.amenity_99,row.amenity_103,row.amenity_105,row.amenity_108,row.amenity_110,
              row.amenity_111, row.amenity_113, row.amenity_121, row.amenity_125, 
              row.veri_1, row.veri_2, row.veri_3, row.veri_4, row.veri_5, row.veri_6,row.veri_7, 
              row.veri_8, row.veri_9, row.veri_10, row.veri_11, row.veri_12, row.veri_13, row.veri_14, row.veri_15, 
              row.veri_16, row.veri_17, row.veri_18, 
              row.veri_19] + one_hot_ng  + one_hot_neighbor + one_hot_bed + one_hot_cancel + one_hot_proper + one_hot_room

    return result

### create features

In [13]:
train_X, train_y = [], []
for (idx, row) in df_pp.iterrows():
    price = float(row.price)
    feature = create_feature(row)

    train_X.append(feature)
    train_y.append(price)

print(len(train_X), len(train_y))

33538 33538


## For whole dataset

In [14]:
catboost_regr = CatBoostRegressor(iterations=1000, # Number of trees (boosting iterations)
                                   learning_rate=0.1, # Step size shrinkage used in update to prevent overfitting
                                   depth=7, # Depth of the trees
                                   loss_function='RMSE', # Loss function to optimize
                                   random_seed=42) 

In [15]:
catboost_regr.fit(train_X, train_y)

0:	learn: 127.5267460	total: 75ms	remaining: 1m 14s
1:	learn: 122.8421194	total: 105ms	remaining: 52.6s
2:	learn: 118.7132436	total: 135ms	remaining: 44.8s
3:	learn: 115.2136470	total: 162ms	remaining: 40.2s
4:	learn: 112.1597002	total: 193ms	remaining: 38.3s
5:	learn: 109.5372054	total: 223ms	remaining: 37s
6:	learn: 107.1986470	total: 253ms	remaining: 35.9s
7:	learn: 105.2054736	total: 283ms	remaining: 35.1s
8:	learn: 103.5492365	total: 309ms	remaining: 34s
9:	learn: 102.0062037	total: 343ms	remaining: 33.9s
10:	learn: 100.6886992	total: 378ms	remaining: 34s
11:	learn: 99.5189688	total: 418ms	remaining: 34.4s
12:	learn: 98.5107869	total: 445ms	remaining: 33.8s
13:	learn: 97.6040374	total: 476ms	remaining: 33.6s
14:	learn: 96.8477967	total: 507ms	remaining: 33.3s
15:	learn: 96.0689812	total: 535ms	remaining: 32.9s
16:	learn: 95.4052254	total: 562ms	remaining: 32.5s
17:	learn: 94.7801296	total: 597ms	remaining: 32.5s
18:	learn: 94.1609395	total: 624ms	remaining: 32.2s
19:	learn: 93.689

In [16]:
y_train_pred = catboost_regr.predict(train_X)
mae = mean_absolute_error(train_y, y_train_pred)
mse = mean_squared_error(train_y, y_train_pred)
r2 = r2_score(train_y, y_train_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Absolute Error: 32.05790573429074
Mean Squared Error: 3042.006330333057
R-squared: 0.8282446721392478


## Test set, without label

In [17]:
import pandas as pd

test_df = pd.read_csv('test.csv')

/tmp/ipykernel_497/3100640438.py:3: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('test.csv')


In [18]:
df_test = test_df.copy()

In [19]:
df_test['host_since_year'] = pd.to_datetime(df_test['host_since']).dt.year
df_test['host_since_month'] = pd.to_datetime(df_test['host_since']).dt.month
df_test['host_since_day'] = pd.to_datetime(df_test['host_since']).dt.day

df_test['first_re_year'] = pd.to_datetime(df_test['first_review']).dt.year
df_test['last_re_year'] = pd.to_datetime(df_test['last_review']).dt.year
df_test['first_re_month'] = pd.to_datetime(df_test['first_review']).dt.month
df_test['last_re_month'] = pd.to_datetime(df_test['last_review']).dt.month
df_test['first_re_day'] = pd.to_datetime(df_test['first_review']).dt.day
df_test['last_re_day'] = pd.to_datetime(df_test['last_review']).dt.day

In [20]:
new_columns_data = {}
for idx, amenity in enumerate(all_amenities, start=1):
    new_columns_data[f'amenity_{idx}'] = df_test['amenities'].apply(lambda x: 1 if amenity in x else 0)

df_test = pd.concat([df_test, pd.DataFrame(new_columns_data)], axis=1)

In [21]:
veri_columns_data = {}
for idx, veri in enumerate(all_verifications, start=1):
    veri_columns_data[f'veri_{idx}'] = df_test['host_verifications'].apply(lambda x: 1 if (isinstance(x, str) and veri in x) 
                                                                         else 0 if pd.isnull(x) else 0)

df_test = pd.concat([df_test, pd.DataFrame(veri_columns_data)], axis=1)

In [22]:
test_ids, test_X = [], []
for (idx, row) in df_test.iterrows():
    feature = create_feature(row)
    test_ids.append(row.id)
    test_X.append(feature)
test_y = catboost_regr.predict(test_X)

In [23]:
print(len(test_y))

17337


In [24]:
output_df = pd.DataFrame()
output_df['Id'] = test_ids
output_df['Predicted'] = test_y
output_df.to_csv('catboost_regression_baseline.csv', index = False)